In [76]:
#COpilot langserve client
from langchain.agents import AgentExecutor, tool
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.tools import StructuredTool


In [77]:
from langserve import RemoteRunnable
host = "http://localhost:8000/"

copilotv=RemoteRunnable(host+"cotalk/")
copilotg=RemoteRunnable(host+"kg/")
copilotrag = RemoteRunnable(host+"rag/")

In [168]:
def rag_server_response(query,host="http://localhost:8000/rag",key="query"):
    import requests 
    import json
    resp = requests.get(host+"?"+key+"="+query)
    if resp.status_code==200:
        return json.loads(resp.text)


#Simple function to
def vector_answers(query:str,qkey="question",chain=copilotv)->list:
    """Gets answers using vector RAG.
    Args:
        query: A string containing the question to ask.
        qkey: A string indicating the key for the question in the input dictionary. Default is "question".
        chain: A RemoteRunnable object that represents the vector RAG model.
    Returns:
        A list of answers from the vector RAG model.
    """
    
    return chain({"question":query}.get("answer"))
    

def RAG_asnwers_tool():
    structured_tool = StructuredTool.from_function(
        name="Get LLM response using vector or graph RAG tool",
        func=copilot_response_server,
        description="Copilot use this tool if you want to get a RAG response using knowledge graph or vector embeddings from business documents by API provisioning LangServe")

    return structured_tool

# from langchain.agents import tool


# @tool
# def get_word_length(word: str) -> int:
#     """Returns the length of a word."""
#     return len(word)


# @tool
# def graph_response(query):
#     pass

async def stream_obj_response(copilotobj,query,qkey="question"):
    "Streaming response using async stream"
    final_response = []
    async for msg in copilotobj.astream({qkey:query}):
        print(msg.get("answer"), end="", flush=True)
        final_response.append(msg.get("answer"), end="", flush=True)
    return final_response
        
    

def copilot_response_server(query:str, qkey="question",fullrag=False,copilotobj=None,stream=False):
    "Gets answers using vector RAG from vectors, graphs or RAG response class"
    if fullrag:
        r = rag_server_response(query)
    else:
        #qkey for vector question and for graph 
        if stream: #streaming response
            r = stream_obj_response(copilotobj,query,qkey)
        else:
            r=copilotobj.invoke({qkey:query})
    return r




## Testing Vector Retriever in Langserve

In [169]:
# import asyncio
# asyncio.run(stream_obj_response(copilotobj=copilotv,query="how many tickets are related to benchmill"))

In [156]:
copilot_response_server("how many tickets are related to benchmill",copilotobj=copilotv)

2024-02-16 07:30:14 [INFO]: HTTP Request: POST http://localhost:8000/cotalk/invoke "HTTP/1.1 200 OK"


{'answer': 'Based on the provided incident tickets, there is one ticket (INC0007654) that is related to the Benchmill.'}

In [157]:
copilot_response_server("what are the others about then",copilotobj=copilotv)

2024-02-16 07:30:17 [INFO]: HTTP Request: POST http://localhost:8000/cotalk/invoke "HTTP/1.1 200 OK"


{'answer': 'The other incident tickets are about a sudden disruption in Production Line 2 due to the unexpected shutdown of Benchmill, excessive vibrations or unusual noise on Production Line 11 leading to poor product quality, a sudden and unplanned production stoppage on Assembly Line 3 impacting the deliverables, and variations in the quality and accuracy of produced machined parts on Assembly Line 7.'}

In [158]:
copilot_response_server("can you perform an individual root cause analysis and give me a plan of action for each of them",copilotobj=copilotv)

2024-02-16 07:30:24 [INFO]: HTTP Request: POST http://localhost:8000/cotalk/invoke "HTTP/1.1 200 OK"


{'answer': 'Sure, I can perform a root cause analysis for each incident ticket and provide a plan of action. Here are the results:\n\nIncident Ticket ID: INC0007654\nRoot Cause Analysis: The unexpected shutdown of Benchmill on Production Line 2 was caused by high motor surface temperature (80 degree celsius) and high load on the motor (70%). This led to overheating and subsequent shutdown.\n\nPlan of Action: \n1. Implement regular maintenance checks to ensure proper cooling of the motor and prevent overheating.\n2. Optimize the load on the motor to reduce the risk of overheating.\n3. Monitor motor surface temperature during operation and take corrective actions if it exceeds the recommended limit.\n\nIncident Ticket ID: INC0003557\nRoot Cause Analysis: The unplanned production stoppage on Assembly Line 3 was caused by excessive dust accumulation (2 mm thickness) on the equipment. This resulted in reduced performance and eventual stoppage.\n\nPlan of Action:\n1. Increase the frequency o

#### Vector Retriever Stream response

In [162]:
#as stream response
async for msg in copilotv.astream({"question":"can you perform an individual root cause analysis for ticket 1"}):
    print(msg.get("answer"), end="", flush=True)

2024-02-16 07:32:37 [INFO]: HTTP Request: POST http://localhost:8000/cotalk/stream "HTTP/1.1 200 OK"


Based on the provided information, the root cause analysis for Incident Ticket ID INC0007654 is as follows:

The unexpected shutdown of Benchmill on Production Line 2 during the morning shift was caused by a combination of factors. The motor surface temperature of 80 degrees Celsius indicates that the motor was operating at a high temperature, potentially leading to overheating. The load on the motor was at 70%, which suggests that the motor was operating close to its maximum capacity. Additionally, the voltage stability was at 5%, indicating fluctuations in the power supply. 

These factors, combined with the monthly maintenance logs showing a lack of regular maintenance, such as dust accumulation of 0.5 mm thickness, may have contributed to the sudden disruption and shutdown of Benchmill. Further analysis is recommended to determine the exact cause and develop appropriate corrective actions.

In [170]:
#stream_obj_response(copilotv,"can you perform an individual root cause analysis for ticket 1",qkey="question")
#from function it yields corutine response
copilot_response_server("perform an individual root cause analysis for ticket 1",copilotobj=copilotv,stream=True)

<coroutine object stream_obj_response at 0x0000028C1DC77740>

## Testing GraphQA Retriever in Langserver

In [171]:
copilot_response_server("what did the study concluded",copilotobj=copilotg)

2024-02-16 07:36:19 [INFO]: HTTP Request: POST http://localhost:8000/kg/invoke "HTTP/1.1 200 OK"


{'result': 'The study concluded that only about 13% of methane emissions are covered by methane mitigation policies. Additionally, the effectiveness of these policies is far from clear.'}

In [172]:
#RUN examples
copilot_response_server("tell me everything you know about",copilotobj=copilotg)


2024-02-16 07:36:23 [INFO]: HTTP Request: POST http://localhost:8000/kg/invoke "HTTP/1.1 200 OK"


{'result': "I know several pieces of information about different topics. Here are some of them:\n\n1. There are theories of planetary formation that have been challenged.\n2. There is a lowest-mass star hosting something, although it is not specified what it is hosting.\n3. Five employees have published a tropical cyclone report.\n4. In August, there was a new record high temperature reached.\n5. The world's oceans have also reached a new record high temperature.\n6. Hurricane Ian was upgraded from Category 4 to Category 5.\n7. Hurricane Ian caused $112.9 billion worth of damage.\n8. Hurricane Ian struck Florida.\n9. The oxygen partial pressure is above 18%.\n10. Emissions of chlorofluorocarbons have risen.\n\nPlease let me know if you would like more information on any specific topic."}

In [173]:
copilot_response_server("tell me about the earths heaviest animal",copilotobj=copilotg)


2024-02-16 07:36:27 [INFO]: HTTP Request: POST http://localhost:8000/kg/invoke "HTTP/1.1 200 OK"


{'result': "The Earth's heaviest animal is the blue whale. It can weigh up to 200 tons, which is equivalent to about 400,000 pounds. These magnificent creatures can grow to be over 100 feet long and have a heart that weighs as much as a car. Despite their massive size, blue whales primarily feed on tiny shrimp-like animals called krill."}

In [174]:
copilot_response_server("what do you know about the root cause analysis",copilotobj=copilotg)

2024-02-16 07:36:30 [INFO]: HTTP Request: POST http://localhost:8000/kg/invoke "HTTP/1.1 200 OK"


{'result': 'Root cause analysis is a systematic process used to identify the underlying causes of a problem or issue. It involves investigating and analyzing the various factors that contribute to the occurrence of the problem, with the goal of identifying the primary cause or causes. By understanding the root cause, organizations can develop effective solutions and prevent the problem from recurring in the future. Root cause analysis is commonly used in industries such as manufacturing, healthcare, and engineering to improve processes, enhance quality, and minimize risks.'}

## RAG responses

In [91]:
#RESPONSES from messages without memory

In [147]:
import requests
import json

In [175]:
#RESPONSE
copilot_response_server("Perform FMEA on first ticket data",fullrag=True)

{'result': 'Based on the data from the first ticket (INC0007654), a Failure Mode and Effects Analysis (FMEA) can be performed to identify potential failure modes, their effects, and the severity, occurrence, and detection ratings associated with each failure mode. However, the provided data does not include information on the specific failure mode or the associated effects. Therefore, without additional information, a comprehensive FMEA cannot be performed.'}

In [176]:
copilot_response_server("Perform RCA on first ticket data",fullrag=True)

{'result': 'Based on the provided data for Incident Ticket ID INC0007654, the RCA can be performed as follows:\n\n1) Check the cooling mechanisms: The airflow rate through the cooling vents is 70 CFM, which is above the standard value of 50 CFM. Cooling mechanisms are adequate.\n\n2) Evaluate ambient temperature around the motor: The ambient temperature is 35 degrees Celsius, which is above the standard value of 20-25 degrees Celsius. The high ambient temperature may contribute to inadequate cooling.\n\n3) Motor surface temperature during operation: The motor surface temperature is 80 degrees Celsius, which is above the standard value of 60 degrees Celsius. The high motor surface temperature indicates inadequate cooling.\n\nBased on the above analysis, the likely root cause of the unexpected shutdown of Benchmill is inadequate cooling leading to motor overheating.'}

In [181]:
copilot_response_server("What was my 2 previous question?",fullrag=True)

{'result': 'Previous Question 1: What could be the root cause of the sudden disruption in Production Line 2 due to the unexpected shutdown of Benchmill?\n\nPrevious Question 2: What could be the root cause of the excessive vibrations or unusual noise on Production Line 11 leading to poor product quality?'}

This RESPONSE has no memory

## Agent executor LLMS

In [182]:
#remote 
from ResponsePipeline import vecrag,llmrca,loaded_embeddings,rag2

In [149]:
#DEPLOYMENT 
#You can deploy to Azure using Azure Container Apps (Serverless):
#az containerapp up --name [container-app-name] --source . --resource-group [resource-group-name] --environment  [environment-name] --ingress external --target-port 8001 --env-vars=OPENAI_API_KEY=your_key


tools_llm = llmrca.bind(functions = [format_tool_to_openai_function(t) for t in tools])


In [ ]:
tools_llm = llmrca.bind(functions = [format_tool_to_openai_function(t) for t in tools])


In [82]:
#LLM RCA WITH TOOLS

tools = [RAG_asnwers_tool]

agent = initialize_agent(llm=llmrca,
                         tools=tools,
                         agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

NameError: name 'initialize_agent' is not defined

In [99]:
vc2=vecrag.get_retriever(method="vector",text_embeddings = loaded_embeddings,store="faiss")

Getting vector retriever


2024-02-15 15:46:12 [INFO]: Loading faiss with AVX2 support.
2024-02-15 15:46:12 [INFO]: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2024-02-15 15:46:12 [INFO]: Loading faiss.
2024-02-15 15:46:12 [INFO]: Successfully loaded faiss.


Building vector chain 


In [100]:
vectorchain2 = vecrag.get_retriever(method="vector",text_embeddings = loaded_embeddings,store="faiss")


Getting vector retriever
Building vector chain 


In [102]:
vc2.invoke({"question":"number of tickets"})

2024-02-15 15:46:30 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/azure_embedding/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
                Given a specific context, please give a short answer to the question talking like a professional Manufacturing AI Assistant.
                Your answers should be concise, precise and should easily demonstrate you are an expert. Remember you are also a specialist in Root Cause Analysis and FMEA
                you can perform RCA and FMEA basing yourself on the reference values retrieved and comparing them to the incidents. 
                ----
                ﻿Incident Ticket ID: INC0004823
Issue Description: Excessive vibrations or unusual noise occurred on Production Line 11 and lead to poor product quality.
Parameter Values: Airflow Rate : 70 CFM, Ambient Temperature : 22 degree celsius, Motor surface temperature : 50 degree celsius, Motor rated power : 1.5 HP,
Load on motor :  90% , User workload : 82%, Voltage stability : 5%, Power wiring resistanc

2024-02-15 15:46:31 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.


{'question': 'number of tickets',
 'chat_history': [HumanMessage(content='number of tickets'),
  AIMessage(content='Based on the given information, there are 4 incident tickets mentioned.')],
 'answer': 'Based on the given information, there are 4 incident tickets mentioned.'}

In [114]:
fres = []
async for n in copilotv.astream(
      {"question": "what tickets are related to bad maintainance"}
):
    fres.append(n)

fres[0]

2024-02-15 16:20:49 [INFO]: HTTP Request: POST http://localhost:8000/cotalk/stream "HTTP/1.1 200 OK"


{'question': 'what tickets are related to bad maintainance',
 'chat_history': [HumanMessage(content='how many tickets are related to benchmill'),
  AIMessage(content='Based on the provided incident tickets, there is 1 ticket related to the Benchmill.'),
  HumanMessage(content='what are the others about then'),
  AIMessage(content='Ticket INC0004823: Excessive vibrations or unusual noise occurred on Production Line 11 and lead to poor product quality.\n\nTicket INC0001198: Variations in the quality and accuracy of produced machined parts on Assembly Line 7.\n\nTicket INC0007654: Production Line 2 experienced a sudden disruption during the morning shift due to unexpected shutdown of Benchmill impacting the production schedule.\n\nTicket INC0003557: A sudden and unplanned production stoppage occurred on Assembly Line 3 impacting the deliverables.'),
  HumanMessage(content='can you perform an individual root cause analysis and give me a plan of action for each of them'),
  AIMessage(conten

In [115]:
fres[0].get("answer")

'Based on the provided information, the incident tickets related to poor maintenance are:\n\n1. Incident Ticket ID: INC0003557 - A sudden and unplanned production stoppage occurred on Assembly Line 3, impacting the deliverables. The incident is related to poor maintenance due to the high dust accumulation of 2 mm thickness, which indicates a lack of regular cleaning and maintenance.\n\n2. Incident Ticket ID: INC0001198 - Variations in the quality and accuracy of produced machined parts on Assembly Line 7. This incident is related to poor maintenance due to the high voltage fluctuation of 4% and the high power wiring resistance of 0.51 ohms, indicating a lack of proper electrical maintenance and monitoring.'

In [143]:
async for chunk in copilotv.astream(
      {"question": "what tickets are related to bad maintainance"}
):
        print(chunk)

2024-02-15 19:42:30 [INFO]: HTTP Request: POST http://localhost:8000/cotalk/stream "HTTP/1.1 200 OK"


{'question': 'what tickets are related to bad maintainance', 'chat_history': [HumanMessage(content='how many tickets are related to benchmill'), AIMessage(content='Based on the provided incident tickets, there is 1 ticket related to the Benchmill.'), HumanMessage(content='what are the others about then'), AIMessage(content='Ticket INC0004823: Excessive vibrations or unusual noise occurred on Production Line 11 and lead to poor product quality.\n\nTicket INC0001198: Variations in the quality and accuracy of produced machined parts on Assembly Line 7.\n\nTicket INC0007654: Production Line 2 experienced a sudden disruption during the morning shift due to unexpected shutdown of Benchmill impacting the production schedule.\n\nTicket INC0003557: A sudden and unplanned production stoppage occurred on Assembly Line 3 impacting the deliverables.'), HumanMessage(content='can you perform an individual root cause analysis and give me a plan of action for each of them'), AIMessage(content='Sure, I 

TESTING WITH DIFFERENT TOOLS

In [84]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [88]:
llm_with_tools = llmrca.bind_tools(tools)



In [90]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [93]:
# list(agent_executor.invoke({"input": "How many letters in the word eudca"}))



In [98]:
llm_with_tools.invoke("How many letters in the word eudca")

2024-02-15 15:38:04 [INFO]: HTTP Request: POST https://usedoai0efaoa03.openai.azure.com//openai/deployments/mxdrca/chat/completions?api-version=2023-05-15 "HTTP/1.1 404 model_error"


NotFoundError: Error code: 404 - {'error': {'message': 'Unrecognized request argument supplied: tools', 'type': 'invalid_request_error', 'param': None, 'code': None}}